<a href="https://colab.research.google.com/github/beesaprathyusha20/Deep-Learning/blob/main/Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [26]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/train_fakenews.csv")

In [27]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [28]:
###Drop Nan Values
df=df.dropna()

In [29]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [30]:
## Get the Dependent features
y=df['label']

In [31]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [32]:
X.shape

(18285, 4)

In [33]:
y.shape

(18285,)

In [34]:
import tensorflow as tf

In [35]:
tf.__version__

'2.9.2'

In [36]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [37]:
### Vocabulary size
voc_size=5000

In [38]:
messages=X.copy()

In [39]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [40]:
messages.reset_index(inplace=True)

In [41]:
import nltk
import re
from nltk.corpus import stopwords

In [42]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
#     print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [44]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1009, 593, 3284, 1336, 3922, 2789, 3034, 134, 1275, 156],
 [1137, 526, 528, 375, 3901, 1448, 1910],
 [4446, 1599, 90, 3311],
 [3667, 3700, 313, 3814, 559, 1735],
 [3149, 3901, 3237, 4700, 242, 3326, 3901, 4626, 1506, 3296],
 [1021,
  2362,
  3895,
  2581,
  2829,
  431,
  3699,
  978,
  3286,
  516,
  3657,
  1283,
  334,
  860,
  1910],
 [13, 3811, 4501, 4523, 4804, 635, 1719, 2036, 1309, 4585, 682],
 [3890, 4289, 4825, 2334, 3689, 2527, 431, 319, 1309, 4585, 682],
 [1162, 2917, 2977, 4521, 2263, 1824, 419, 729, 431, 333],
 [3827, 3186, 1123, 814, 72, 2160, 773, 4198],
 [591, 3963, 3629, 408, 1185, 2999, 2344, 1748, 3158, 4414, 4566],
 [3814, 3423, 3922, 1824, 431, 3689],
 [1758, 594, 1994, 234, 1921, 887, 978, 2453, 3597],
 [3195, 123, 3089, 3179, 541, 3992, 1986, 1309, 4585, 682],
 [4839, 1889, 4772, 4717, 476, 1309, 4585, 682],
 [2421, 4942, 1836, 4763, 4209, 1761, 4841, 393, 1834, 2610],
 [1572, 2882, 526],
 [4956, 3732, 2289, 1289, 431, 756, 187, 1910],
 [1858, 3991, 528, 2410,

In [45]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  134 1275  156]
 [   0    0    0 ... 3901 1448 1910]
 [   0    0    0 ... 1599   90 3311]
 ...
 [   0    0    0 ... 1309 4585  682]
 [   0    0    0 ... 4989  735 4664]
 [   0    0    0 ... 4580  492  949]]


In [46]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1009,
        593, 3284, 1336, 3922, 2789, 3034,  134, 1275,  156], dtype=int32)

In [47]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [49]:
len(embedded_docs),y.shape

(18285, (18285,))

In [50]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [51]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [53]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 14ms/step - loss: 0.3129 - accuracy: 0.8495 - val_loss: 0.2008 - val_accuracy: 0.9152
Epoch 2/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1421 - accuracy: 0.9461 - val_loss: 0.2011 - val_accuracy: 0.9176
Epoch 3/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0981 - accuracy: 0.9663 - val_loss: 0.2388 - val_accuracy: 0.9135
Epoch 4/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0702 - accuracy: 0.9776 - val_loss: 0.2672 - val_accuracy: 0.9137
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0448 - accuracy: 0.9863 - val_loss: 0.2937 - val_accuracy: 0.9195
Epoch 6/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0324 - accuracy: 0.9897 - val_loss: 0.3961 - val_accuracy: 0.9115
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0199 - accuracy: 0.9949 - val_loss: 0.4294 - val_accuracy: 0.912

In [54]:
y_pred1=model1.predict(X_test)
classes_y=np.argmax(y_pred1,axis=1)

189/189 [==============================] - 2s 3ms/step


In [55]:
from sklearn.metrics import confusion_matrix

In [56]:
confusion_matrix(y_test,classes_y)

array([[3419,    0],
       [2616,    0]])

In [57]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,classes_y)

0.5665285832642917

In [58]:
from sklearn.metrics import classification_report
print(classification_report(y_test,classes_y))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
